# Baselines

In [1]:
import itertools
import mantrap
import mantrap_evaluation
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm

### Evaluation Configuration

In [2]:
max_time_steps = 10
num_tests = 1

### Testing Scenarios

In [3]:
config_num_ados = [10, 8, 4, 2]
config_num_scenarios = [int(np.random.rand(1)) * 9999 for _ in range(5)]

environments = [mantrap.environment.PotentialFieldEnvironment, 
                mantrap.environment.Trajectron]

scenarios_df = pd.DataFrame()
for num_ados, i, env_type in itertools.product(*[config_num_ados, config_num_scenarios, environments]):
    torch.manual_seed(i)
    env, goal, _ = mantrap_evaluation.scenarios.random(env_type, num_ados=num_ados)
    scenarios_df = scenarios_df.append({"num_ados": num_ados, "scenario": i, "env_name": env.name, 
                                        "env": env, "goal": goal}, ignore_index=True)

print(scenarios_df.shape)
scenarios_df.head()

(40, 5)


,env,env_name,goal,num_ados,scenario
0,<mantrap.environment.simplified.potential_fiel...,potential_field,"[tensor(-8.2305), tensor(-7.3594)]",10.0,0.0
1,<mantrap.environment.trajectron.Trajectron obj...,trajectron,"[tensor(-8.2305), tensor(-7.3594)]",10.0,0.0
2,<mantrap.environment.simplified.potential_fiel...,potential_field,"[tensor(-8.2305), tensor(-7.3594)]",10.0,0.0
3,<mantrap.environment.trajectron.Trajectron obj...,trajectron,"[tensor(-8.2305), tensor(-7.3594)]",10.0,0.0
4,<mantrap.environment.simplified.potential_fiel...,potential_field,"[tensor(-8.2305), tensor(-7.3594)]",10.0,0.0


### Solver configuration

In [4]:
solvers = [(mantrap.solver.baselines.RRTStarSolver, "rrt_star"),
           (mantrap.solver.baselines.ORCASolver, "orca"),
           (mantrap.solver.baselines.MonteCarloTreeSearch, "mcts"), 
           (mantrap.solver.IPOPTSolver, "ipopt")]

modules = [(None, "default"),  # default
           ([mantrap.modules.baselines.InteractionEllipsoidModule, 
            mantrap.modules.ControlLimitModule, mantrap.modules.GoalNormModule], "ellipsoidal"),
           ([mantrap.modules.baselines.InteractionPositionModule, 
            mantrap.modules.ControlLimitModule, mantrap.modules.GoalNormModule], "inter_pos")]

warm_starts = [None,   # default - warm_start_hard
               mantrap.constants.WARM_START_POTENTIAL,
               mantrap.constants.WARM_START_ZEROS]

attentions = [(mantrap.attention.ClosestModule, "closest"), 
              (mantrap.attention.EuclideanModule, "euclidean"),
              (None, "none")]  # default

max_cpu_runtimes = [0.5, 1.0, 2.0]

In [5]:
configs_df = pd.DataFrame()
combined = [solvers, modules, warm_starts, attentions, max_cpu_runtimes]
for solver, module, warm_start, attention, max_cpu_time in itertools.product(*combined):
    solver, solver_name = solver
    module, module_name = module
    attention, attention_name = attention
    
    if solver != mantrap.solver.IPOPTSolver and any([module, warm_start, attention]):
        continue
    
    configs_df = configs_df.append({"solver_type": solver, "solver_name": solver_name, 
                                    "modules": module, "modules_name": module_name, 
                                    "attention": attention, "attention_name": attention_name,
                                    "warm_start": warm_start, "max_cpu_time": max_cpu_time}, 
                                    ignore_index=True)
print(configs_df.shape)
configs_df.head()

(90, 8)


,attention,attention_name,max_cpu_time,modules,modules_name,solver_name,solver_type,warm_start
0,None,none,0.5,None,default,rrt_star,<class 'mantrap.solver.baselines.rrt_star.RRTS...,None
1,None,none,1.0,None,default,rrt_star,<class 'mantrap.solver.baselines.rrt_star.RRTS...,None
2,None,none,2.0,None,default,rrt_star,<class 'mantrap.solver.baselines.rrt_star.RRTS...,None
3,None,none,0.5,None,default,orca,<class 'mantrap.solver.baselines.orca.ORCASolv...,None
4,None,none,1.0,None,default,orca,<class 'mantrap.solver.baselines.orca.ORCASolv...,None


### Running evaluation 

In [ ]:
eval_df = pd.DataFrame()
for _, sdf in tqdm(scenarios_df.iterrows(), total=len(scenarios_df.index)):
    scenario_id = sdf["scenario"]
    torch.manual_seed(scenario_id)
    
    for _, cdf in tqdm(configs_df.iterrows(), total=len(configs_df.index)):
        solver_obj = cdf["solver_type"]
        env_obj = sdf["env"]

        # try:
        solver = solver_obj(**cdf.to_dict(), **sdf.to_dict())
        eval_df_k, _, _ = mantrap_evaluation.evaluate(solver, mean_df=False, **sdf.to_dict(), **cdf.to_dict(),
                                                      max_time_steps=max_time_steps, num_tests=num_tests)

        for _, eval_df_ki in eval_df_k.iterrows():
            eval_df = eval_df.append(pd.concat([eval_df_ki, cdf.copy(), sdf.copy()]), ignore_index=True)
        eval_df.to_pickle("eval_data.pkl")

        # except:
        #     pass